In [19]:
import pandas as pd
import tensorflow as tf

# Automatically reload imported modules that are changed outside this notebook
# More pixels in figures
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200


In [20]:
import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
dev = pd.read_csv("dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
#test = test.sample(30000, replace=False)
meta = pd.concat([train, test, dev])


In [21]:
test.shape

(55768, 4)

In [22]:
meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)

workdir = "/tf/datasets/xvector"
def to_1hot(x):
    a = [0, 0, 0, 0]
    a[x] = 1
    return a

#meta["target"] = meta["target"].apply(to_1hot)


In [23]:
meta

,Unnamed: 0,path,locale,split,target,id
0,1486,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1,1486
1,56701,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,train,0,56701
2,3364,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,train,1,3364
3,110475,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,rw,train,3,110475
4,45384,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,train,2,45384
...,...,...,...,...,...,...
30751,2677,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,1,2677
30752,881,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,dev,1,881
30753,68709,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,dev,0,68709
30754,249025,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,en,dev,2,249025


In [24]:
import scipy.signal

from lidbox.features import audio, cmvn
import lidbox.data.steps as ds_steps


TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def metadata_to_dataset_input(meta):
    return {
        "id": tf.constant(meta.id, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }

def read_mp3(x):
    s, r = audio.read_mp3(x["path"])
    out_rate = 16000
    s = audio.resample(s, r, out_rate)
    s = audio.peak_normalize(s, dBFS=-3.0)
    s = audio.remove_silence(s, out_rate)
    return dict(x, signal=s, sample_rate=out_rate)


def random_filter(x):
    def scipy_filter(s, N=10):
        b = np_rng.normal(0, 1, N)
        return scipy.signal.lfilter(b, 1.0, s).astype(np.float32), b
    s, _ = tf.numpy_function(
        scipy_filter,
        [x["signal"]],
        [tf.float32, tf.float64],
        name="np_random_filter")
    s = tf.cast(s, tf.float32)
    s = audio.peak_normalize(s, dBFS=-3.0)
    return dict(x, signal=s)


def random_speed_change(ds):
    return ds_steps.random_signal_speed_change(ds, min=0.9, max=1.1, flag=None)


def create_signal_chunks(ds):
    ds = ds_steps.repeat_too_short_signals(ds, 3200)
    ds = ds_steps.create_signal_chunks(ds, 3200, 800)
    return ds


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        S = audio.spectrograms(signals, rates[0])
        S = audio.linear_to_mel(S, rates[0])
        S = tf.math.log(S + 1e-6)
        S = cmvn(S, normalize_variance=False)
    return dict(x, logmelspec=S)

def pipeline_from_meta(data, split):
    if split == "train":
        data = data.sample(frac=1, random_state=np_rng.bit_generator)

    ds = (tf.data.Dataset
            .from_tensor_slices(metadata_to_dataset_input(data))
            .map(read_mp3, num_parallel_calls=TF_AUTOTUNE))

    if split == "train":
        return (ds
            .apply(random_speed_change)
           #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1)
            .map(random_filter, num_parallel_calls=TF_AUTOTUNE)
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch())
    else:
        return (ds
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch()
            #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1))


cachedir = os.path.join(workdir, "cache")

split2ds = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}

2021-06-02 11:26:05.074 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.900, 1.100]
2021-06-02 11:26:05.243 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 11:26:05.272 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-02 11:26:06.118 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 11:26:06.142 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-02 11:26:06.987 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 11:26:07.013 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

In [25]:
import lidbox.models.xvector as xvector
model_input_type = "logmelspec"

def create_model(num_freq_bins, num_labels):
    model = xvector.create([None, num_freq_bins], num_labels, channel_dropout_rate=0.8)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
        metrics=tf.keras.metrics.sparse_categorical_accuracy)
    return model


model = create_model(
    num_freq_bins=20 if model_input_type == "mfcc" else 40,
    num_labels=4)
model.summary()

callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(cachedir, "tensorboard", model.name),
        update_freq="epoch",
        write_images=True,
        profile_batch=0,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(cachedir, "model", model.name),
        monitor='val_loss',
        save_weights_only=True,
        save_best_only=True,
        verbose=1,
    ),
]


def as_model_input(x):
    return x["logmelspec"], x["target"]



callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(cachedir, "tensorboard", model.name),
        update_freq="epoch",
        write_images=True,
        profile_batch=0,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(cachedir, "model", model.name),
        monitor='val_loss',
        save_weights_only=True,
        save_best_only=True,
        verbose=1,
    ),
]


def as_model_input(x):
    return x["logmelspec"], x["target"]



Model: "x-vector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None, 40)]        0         
_________________________________________________________________
channel_dropout (SpatialDrop (None, None, 40)          0         
_________________________________________________________________
frame1 (Conv1D)              (None, None, 512)         102912    
_________________________________________________________________
frame2 (Conv1D)              (None, None, 512)         786944    
_________________________________________________________________
frame3 (Conv1D)              (None, None, 512)         786944    
_________________________________________________________________
frame4 (Conv1D)              (None, None, 512)         262656    
_________________________________________________________________
frame5 (Conv1D)              (None, None, 1500)        769

In [26]:
meta = meta.set_index("id")

In [27]:
import pandas as pd

from lidbox.util import predict_with_model, classification_report
from lidbox.visualize import draw_confusion_matrix


def load_trained_model():
    model = create_model()
    print(os.path.join(cachedir, "model", model.name))
    model.load_weights(os.path.join(cachedir, "model", model.name))
    return model


def display_classification_report(report):
    for m in ("avg_detection_cost", "avg_equal_error_rate", "accuracy"):
        print("{}: {:.3f}".format(m, report[m]))

    lang_metrics = pd.DataFrame.from_dict(
        {k: v for k, v in report.items() if k in lang2target})
    lang_metrics["mean"] = lang_metrics.mean(axis=1)
    display(lang_metrics.T)

    fig, ax = draw_confusion_matrix(report["confusion_matrix"], lang2target)

model.load_weights("xvector.h5")

def predict_with_ap_loss(x):
    with tf.device("GPU"):
        # Generate language vector for input spectra
        language_vector = model(x["input"], training=False)
        print(language_vector)
        # Predict languages by computing distances to reference directions
        return x["id"], model.loss.predict(language_vector)




In [28]:

def predictions_to_dataframe(ids, predictions):
    return (pd.DataFrame.from_dict({"id": ids, "prediction": predictions})
            #.set_index("id", drop=True, verify_integrity=True)
            #.sort_index()
           )

def predict_with_model(model, ds, predict_fn=None):
    """
    Map callable model over all batches in ds, predicting values for each element at key 'input'.
    """
    if predict_fn is None:
        def predict_fn(x):
            with tf.device("GPU"):
                return x["id"], model(x["input"], training=False)

    ids = []
    predictions = []
    for id, pred in ds.map(predict_fn, num_parallel_calls=TF_AUTOTUNE).unbatch().as_numpy_iterator():
        ids.append(id.decode("utf-8"))
        predictions.append(pred)

    return predictions_to_dataframe(ids, predictions)

In [29]:
chunk2pred = predict_with_model(
    model=model,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [31]:
chunk2pred

,id,prediction
0,71684-000001,"[-9.007315, -2.5206907, -0.30927205, -1.6847415]"
1,71684-000002,"[-7.734203, -3.8458073, -0.25859794, -1.5795889]"
2,71684-000003,"[-9.240495, -2.5607886, -0.6035977, -0.9786491]"
3,71684-000004,"[-7.389434, -2.4304147, -0.48849627, -1.2112205]"
4,88574-000001,"[-0.5346573, -2.4032218, -2.269483, -1.5125916]"
...,...,...
175433,259-000003,"[-9.565975, -0.30343857, -5.014595, -1.3664354]"
175434,12245-000001,"[-9.821604, -0.85450804, -1.3720167, -1.136756]"
175435,12245-000002,"[-5.950877, -1.2592, -1.6123981, -0.66532457]"
175436,12245-000003,"[-6.741986, -1.714731, -2.277125, -0.3337555]"


In [32]:
backup = chunk2pred.copy()

In [33]:
chunk2pred = chunk2pred.set_index("id")

In [34]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
0,"[-6.246306, -1.2479299, -2.071201, -0.7565291]"
1,"[-6.414607, -1.6929551, -2.1039782, -0.6282889]"
100,"[-8.636872, -1.0326728, -2.232252, -0.73449343]"
1000,"[-5.0780663, -1.2752019, -2.4993756, -0.78996176]"
10000,"[-8.583008, -3.0993524, -1.3305961, -0.3704628]"
...,...
99974,"[-0.09855579, -4.1048737, -4.31477, -2.9244318]"
99983,"[-0.10056982, -4.5823064, -6.5588746, -3.9748685]"
99986,"[-0.039884023, -4.4768705, -6.8049088, -3.880857]"


In [35]:
from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.95      0.86      0.90     17341
          ru       0.78      0.47      0.58     10379
          en       0.95      0.69      0.80     12964
       other       0.55      0.89      0.68     15084

    accuracy                           0.76     55768
   macro avg       0.81      0.73      0.74     55768
weighted avg       0.81      0.76      0.76     55768



In [36]:
import sklearn
sklearn.metrics.confusion_matrix(y_true=true_sparse, y_pred=pred_dense)

array([[14911,   217,    79,  2134],
       [  265,  4838,    54,  5222],
       [  162,   213,  8981,  3608],
       [  416,   959,   304, 13405]])

In [130]:
len(list(meta.loc[meta["split"] == "test"].index))

20000

In [43]:
test_meta = test_meta.set_index("Unnamed: 0")

In [46]:
test_meta 

,path,locale,split,target,prediction
Unnamed: 0,,,,,
0.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,zh-CN,test,3.0,"[-1.5906578, -1.5723495, -1.5997407, -1.5868733]"
0.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,test,1.0,"[-1.5906578, -1.5723495, -1.5997407, -1.5868733]"
1.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,test,1.0,"[-1.5859909, -1.5551229, -1.5327619, -1.5346978]"
1.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ky,test,3.0,"[-1.5859909, -1.5551229, -1.5327619, -1.5346978]"
1.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,zh-CN,test,3.0,"[-1.5859909, -1.5551229, -1.5327619, -1.5346978]"
...,...,...,...,...,...
99935.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,es,test,3.0,"[-1.658407, -1.5379996, -1.5907953, -1.5804672]"
9994.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,ru,test,1.0,"[-1.6105947, -1.5339103, -1.6226405, -1.5670581]"
99958.0,/tf/datasets/data_untar/cv-corpus-6.1-2020-12-...,kz,test,0.0,"[-1.5755433, -1.5537702, -1.547276, -1.5391484]"


In [16]:
chunk2pred = predict_with_model(
    model=new_net,
    ds=split2ds["test"].map(lambda x: dict(x, input=x["logmelspec"])).batch(32),
    #predict_fn=predict_with_ap_loss
    )



In [20]:
chunk2pred

,prediction
id,
71684-000001,"[0.00038267265, 0.068015866, 0.46493787, 0.466..."
71684-000002,"[0.00036228108, 0.030282797, 0.38672042, 0.582..."
71684-000003,"[0.00016482621, 0.031329192, 0.3511263, 0.6173..."
71684-000004,"[1.9100436e-05, 0.0064635505, 0.75075495, 0.24..."
88574-000001,"[1.0, 3.066749e-09, 2.8158553e-09, 1.636421e-08]"
...,...
259-000003,"[4.999837e-08, 0.9930757, 5.8501515e-05, 0.006..."
12245-000001,"[0.00031434407, 0.11699918, 0.03570789, 0.8469..."
12245-000002,"[4.4270102e-05, 0.061910614, 0.0126767345, 0.9..."


In [19]:
chunk2pred = chunk2pred.set_index("id")

KeyError: "None of ['id'] are in the columns"

In [21]:
from lidbox.util import merge_chunk_predictions


utt2pred = merge_chunk_predictions(chunk2pred)
utt2pred

,prediction
id,
0,"[0.0005371941, 0.28633153, 0.069509454, 0.6436..."
1,"[0.00015652769, 0.20875472, 0.12081046, 0.6702..."
100,"[4.5013534e-05, 0.69266313, 0.006293945, 0.300..."
1000,"[0.21455875, 0.36390203, 0.076663665, 0.34487557]"
10000,"[1.0598835e-06, 0.0009105328, 0.0023490405, 0...."
...,...
99974,"[0.9999947, 7.870432e-07, 1.0888429e-07, 4.364..."
99983,"[0.9999991, 5.2881205e-08, 4.136213e-08, 8.014..."
99986,"[0.99999946, 3.9046526e-08, 8.0310656e-08, 5.0..."


In [22]:
from sklearn.metrics import classification_report

test_meta = meta[meta["split"]=="test"].join(utt2pred, how="outer")
assert not test_meta.isna().any(axis=None), "failed to join predictions"

true_sparse = test_meta.target.to_numpy(np.int32)
pred_dense = np.stack(test_meta.prediction.apply(np.argmax))

report = classification_report(true_sparse, pred_dense, target_names=list(targets.keys()), labels=range(4))
print(report)



              precision    recall  f1-score   support

          kz       0.88      0.94      0.91     17341
          ru       0.70      0.64      0.67     10379
          en       0.93      0.82      0.87     12964
       other       0.70      0.74      0.72     15084

    accuracy                           0.80     55768
   macro avg       0.80      0.79      0.79     55768
weighted avg       0.81      0.80      0.80     55768



In [23]:
import sklearn
sklearn.metrics.confusion_matrix(y_true=true_sparse, y_pred=pred_dense)

array([[16319,   359,    96,   567],
       [  800,  6676,   188,  2715],
       [  318,   345, 10665,  1636],
       [ 1084,  2197,   577, 11226]])